# Training an ANN with fewer synops
As described in [Sorbaro et al. 2020](https://www.frontiersin.org/articles/10.3389/fnins.2020.00662/full), it can be beneficial for power consumption and latency in the network to reduce the number of synaptic operations (synops). This number is essentially the output activation multiplied by the number of outward connections (fan-out) to the next layer. We describe how using Sinabs' synops counters it's possible to easily add a term to your loss function which then can be minimised.

Let's start by defining our ANN. Keep in mind that we use NeuromorphicRelus here as we need to discretize the output in the forward pass to simulate the number of spikes that layer would emit. In the backward pass the derivative of the ReLU function is used. 

In [1]:
import torch
import torch.nn as nn
import sinabs
import sinabs.layers as sl


ann = nn.Sequential(
    nn.Conv2d(1, 16, 5, bias=False),
    sl.NeuromorphicReLU(),
    sl.SumPool2d(2),
    nn.Conv2d(16, 32, 5, bias=False),
    sl.NeuromorphicReLU(),
    sl.SumPool2d(2),
    nn.Conv2d(32, 120, 4, bias=False),
    sl.NeuromorphicReLU(),
    nn.Flatten(),
    nn.Linear(120, 10, bias=False),
)

Let's apply a SynopsCounter to our ANN to track how many synaptic operations it would need in an SNN.

In [2]:
batch_size = 5

synops_counter_ann = sinabs.SynOpCounter(ann.modules(), sum_activations=True)

rand_input = torch.rand((batch_size, 1, 28, 28)) * 10
ann(rand_input)
print(f"Synops after feeding input: {synops_counter_ann()}")

Synops after feeding input: 13311.7998046875
